# TP BLOC 5 : application des réseaux récurrents à la traduction de chaines de caractères

**Objectif** : des prénoms, représentés sous la forme de chaines de caractères, ont été encryptés par un algorithme inconnu.

Trois jeux de données vous ont fournis dans le dossier `./6_data_lstm_transform` (train/test set 1,2,3). Les jeux de données sont des ensembles de prénoms "d'entrainement" pour lesquels vous disposerez de la chaine de caractère cryptée et de la chaine de caractères décryptée. Pour chacun des jeux d'entraînement, la fonction de chiffrement est différente.

Vous devrez apprendre un modèle à base de LSTM capable de décrypter un jeu de prénoms de test pour lequel seul la version cryptée est fournie. Pour les besoins du TP, les chaines de caractères décryptées vous sont également fournies pour le jeu de test afin que vous puissiez vérifier le bon fonctionnement de votre modèle.

---



---



### 1. récupération des données

Les séquences de chaines de caractères sont contenus dans deux fichiers, l'un avec les données d'entrainement, l'autre avec celles de test.

Pour chacun, deux tenseurs sont fournis : le premier contient les séquences 'source' qui sont les séquences cryptées, l'autre les séquences 'target' qui sont décryptés. 

Nous vous donnons également une fonction qui permet de transformer les séquences de chiffres en séquences de lettres, plus facile à lire pour les humains.

Toutes les séquences ont été ramenées à 16 caractères par padding. Vous remarquerez aussi qu'il y a un caractère spécial qui marque le début de la chaine.

En regardant les paires de séquences cryptées/décryptées, essayez de comprendre quel code est utilisé pour le cryptage.

En quoi cela justifie-t-il l'usage de réseaux de neurones prenant en compte des données séquentielles ?

In [ ]:
# IMPORTS

import torch
import torch.nn as nn

import pathlib

Si vous utilisez **google colab** :

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# !ls 'gdrive/My Drive/Colab Notebooks/formation IA niveau 3/module 7 advanced ML/Day2'

Dans tous les cas, **définir le fichier où se trouvent les données**

In [ ]:
cur_dir = './6_data_lstm_transformers'

**Extraction des données et visualisation**

Description des données :
- Les données sont des séquences d'entier qui représentent des lettres en format ascii. La fonction `ascii2str` permet de visualiser le texte correspondant
- Pour chaque dataset, on extrait :
    - Les données source (src) qui correspondent au texte chiffré
    - Les données cible/target (tgt) qui correspondent au texte déchiffré que l'on cherche à prédire
    - `num_codes` : correspond au nombre de caractères différents qui peuvent apparaître dans les séquences 

In [ ]:
src_train,tgt_train, num_codes = torch.load(cur_dir+'/'+'train_set.pt', weights_only=True)
src_test,tgt_test, num_codes = torch.load(cur_dir+'/'+'test_set.pt', weights_only=True)
seq_len = tgt_train.shape[1]

print(f"Num codes={num_codes}")
print("taille des tenseurs pour l'entrainement", tgt_train.shape,src_train.shape)
print("taille des tenseurs pour le test", tgt_test.shape,src_test.shape)
print("longueur des séquences : ",seq_len)
def ascii2str(x):
  string =  ''.join([chr(int(i)+ord('a')) if (int(i)+ord('a'))!=ord('z')+2 else '.'  for i in x])
  new_string = string.replace("{", "-" )
  return new_string

for i in range(20):
  print('source : [',ascii2str(src_train[i,:]),"] -> target : [",ascii2str(tgt_train[i,:]),']')



### 2. Construction d'un dataset pytorch

Vous allez désormais construire un objet de type 'torch.utils.data.Dataset' dataset compatible avec les "data loader" (torch.utils.data.DataLoader) de pytorch. 

Les dernières instructions de la cellule (celles qui vous sont fournies) vous permettra de vérifier que ce que vous avez fait fonctionne.

In [ ]:
"""
à vous d'écrire cette classe

class CustomDataset(torch.utils.data.Dataset):
...
"""

class CustomDataset(torch.utils.data.Dataset):
	def __init__(self):
		pass
		
	def __getitem__(self, idx):
		pass
	def __len__(self):
		pass

trainset = CustomDataset(src_train,tgt_train)
testset = CustomDataset(src_test,tgt_test)

train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=16, shuffle=True)

for x_src, x_tgt in test_loader:
	for i in range(x_src.shape[0]):
		print(ascii2str(x_src[i]), '->', ascii2str(x_tgt[i]))
	break

### 3. Construction du modèle à base de LSTM

Vous construirez ensuite le modèle de LSTM qui permet de faire la traduction de chaines de *caractères*.

Le modèle prend en entrée une chaîne de caractère chiffrée et doit prédire le texte déchiffré correspondant

Par la suite, vous pourrez par exemple, **représenter les caractères par un embedding de taille 20**.

In [ ]:
"""
à vous d'écrire le modèle

class mymodel(torch.nn.Module):
"""

class mymodel(torch.nn.Module):
    def __init__(self):
        pass


    def forward(self, x_src):
        pass

### 4. Entrainement du modèle


In [ ]:
"""
à vous d'entrainer le modèle
"""


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device=}")

embedding_size=20
learning_rate = 1e-3
model = mymodel(num_codes=num_codes, embedding_size=32, hidden_size = 256, num_layers = 2).to(device)
model.training = True
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)   


NUM_EPOCH=50

criterion = torch.nn.CrossEntropyLoss()

def train_model(model, nb_epochs ...):
    
	for epoch in range(nb_epochs):
		global_loss=0
		x_prediction = ...
		if epoch%10==0:
			print('epoch: ',epoch, 'loss: ',global_loss)
			print("exemple de décodage d'une donnée de train : ",ascii2str(x_tgt[0,:]),' -> ',ascii2str(torch.argmax(x_prediction,axis=2)[0]),'<fin>')
			my_file = pathlib.Path(f'./model_lstm_epoch{epoch}.pt')
			torch.save({'optimizer':optimizer.state_dict(), 'model':model.state_dict()}, my_file)

### 5. Décodage du jeu de test



In [ ]:
"""
à vous de faire les inférences sur le jeu de test
"""

def test_model(model):
    model.training = False
    with torch.no_grad():
        ...
        x_pred_decoded = ...
        loss_eval = ...
        print("décodage de [", ascii2str(x_src[i]),"] -> [",ascii2str(x_pred_decoded[i]) ,"] GT = ",ascii2str(x_tgt[i]), "Loss Value=", loss_eval)


### 6. Essayer avec les autres jeux de données

Maintenant que l'on a entrainé un modèle à déchiffré des données textuelles, essayez votre modèle avec les autres jeux de données.
- Le modèle 1 fonctionne t'il avec les autres modèles ?
- Entrainez des nouveaux modèles sur les autres jeux de données et comparez les performances des modèles
- Des trois modèles, lequel performe le mieux ? Arrivez-vous à trouver de façon analytique quelle est la fonction de chiffrement utilisée dans les modèles 2 et 3 ?